In [2]:
#下載csv檔
import requests
csv_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
res=requests.get(url=csv_url)
if res.ok:
    print('下載成功')
else:
    print('下載失敗')

下載成功


In [ ]:
import io
import csv
csv_str:str=res.text #csv_str是包含csv資料的字串
csv_file=io.StringIO(csv_str) #將csv_str轉為類文件對象
dict_reader=csv.DictReader(csv_file) #將每行的資料轉為dict
#print(type(dict_reader))
csv_data=list(dict_reader) #將資料轉為list
for row in csv_data: #印出資料
    print(row)
#檢查資料類型
#保證一定有資料
#過濾dict的key